In [ ]:
import jax.numpy as np
from jax import grad, jit, vmap
from jax import random
from jax import jacfwd, jacrev
from jax.numpy import linalg

from numpy import nanargmin,nanargmax 

key = random.PRNGKey(42)